In [8]:
import requests
from bs4 import BeautifulSoup

In [2]:
r = requests.get("https://spotifycharts.com/regional/us/daily/latest")

In [5]:
r.headers["Content-Type"]

'text/html; charset=UTF-8'

In [46]:
html = BeautifulSoup(r.text, 'html.parser')

In [73]:
title_artist = html.find_all('td', attrs = {'class':'chart-table-track'})
streams = html.find_all('td', attrs = {'class':'chart-table-streams'})
ids = html.find_all('td', attrs = {'class':'chart-table-image'})
ids = [i.a.get("href").replace("https://open.spotify.com/track/", "") for i in ids]

In [89]:
t = [{'title':i.strong.text,
      'artist':i.span.text.replace("by ", ""),
      'streams':int(j.text.replace(',', '')),
      'id':ids} for i, j, ids in zip(title_artist, streams, ids)]

# "Stream" will look like this

In [79]:
from time import sleep
for song in t:
    print(song)
    sleep(1)

{'title': 'Sucker', 'artist': 'Jonas Brothers', 'streams': 1340990, 'id': '4y3OI86AEP6PQoDE6olYhO'}
{'title': '7 rings', 'artist': 'Ariana Grande', 'streams': 1282396, 'id': '6ocbgoVGwYJhOv1GgI9NsF'}
{'title': 'MIDDLE CHILD', 'artist': 'J. Cole', 'streams': 1280463, 'id': '2JvzF1RMd7lE3KmFlsyZD8'}
{'title': 'Robbery', 'artist': 'Juice WRLD', 'streams': 1280208, 'id': '6Hj9jySrnFppAI0sEMCZpJ'}
{'title': 'Sunflower - Spider-Man: Into the Spider-Verse', 'artist': 'Post Malone', 'streams': 1260751, 'id': '3KkXRkHbMCARz0aVfEt68P'}
{'title': 'Murder On My Mind', 'artist': 'YNW Melly', 'streams': 1240077, 'id': '7eBqSVxrzQZtK2mmgRG6lC'}
{'title': 'Wow.', 'artist': 'Post Malone', 'streams': 1190389, 'id': '6MWtB6iiXyIwun0YzU6DFP'}
{'title': 'Fast', 'artist': 'Juice WRLD', 'streams': 1129927, 'id': '6MjNilJCVKJq0LfFvo5jWJ'}
{'title': "break up with your girlfriend, i'm bored", 'artist': 'Ariana Grande', 'streams': 1128182, 'id': '4kV4N9D1iKVxx1KLvtTpjS'}
{'title': 'Going Bad (feat. Drake)', 'ar

KeyboardInterrupt: 

# Scraping Function

In [97]:
import requests
from bs4 import BeautifulSoup
from time import sleep

def get_chart(region = 'global', date = 'latest'):
    
    # create url to query
    url = "https://spotifycharts.com/regional/{}/daily/{}".format(region, date)
    
    # run query, if fails, re-try after 3 seconds
    r = requests.get(url)
    while r.status_code != 200:
        print("Error parsing {}\nError code: {}\nTrying again...\n".format(url, r.status_code))
        sleep(5)
        r = requests.get(url)
    
    # Parse HTML response for data
    html = BeautifulSoup(r.text, 'html.parser')
    title_artist = html.find_all('td', attrs = {'class':'chart-table-track'})
    streams = html.find_all('td', attrs = {'class':'chart-table-streams'})
    ids = html.find_all('td', attrs = {'class':'chart-table-image'})
    ids = [i.a.get("href").replace("https://open.spotify.com/track/", "") for i in ids]
    
    # Create a list of dictionaries with the data
    dta =[{'title':i.strong.text,
          'artist':i.span.text.replace("by ", ""),
          'streams':int(j.text.replace(',', '')),
          'id':ids,
          'date':date,
          'region':region} for i, j, ids in zip(title_artist, streams, ids)]
    
    return dta

In [98]:
data = get_chart(region = 'fr', date = "2017-10-19")
for i in data[0:10]:
    print(i)
    sleep(1)

{'title': 'Mwaka Moon', 'artist': 'Kalash', 'streams': 352479, 'id': '1vVZLmZq8gN8pPCEBLjPvD', 'date': '2017-10-19', 'region': 'fr'}
{'title': 'Salé', 'artist': 'Niska', 'streams': 208775, 'id': '0uBu60Wd7yzm2sFrj2hSr6', 'date': '2017-10-19', 'region': 'fr'}
{'title': 'Réseaux', 'artist': 'Niska', 'streams': 192524, 'id': '2nY6MRHWpW0AGXMoGDJOjX', 'date': '2017-10-19', 'region': 'fr'}
{'title': 'Tuba Life', 'artist': 'Niska', 'streams': 158810, 'id': '2r6C2IEw533mDs3sam5Zxx', 'date': '2017-10-19', 'region': 'fr'}
{'title': 'rockstar', 'artist': 'Post Malone', 'streams': 144022, 'id': '7wGoVu4Dady5GV0Sv4UIsx', 'date': '2017-10-19', 'region': 'fr'}
{'title': 'Medellín', 'artist': 'Niska', 'streams': 139880, 'id': '1gI6FedOWXQ7aSj4FfEiGK', 'date': '2017-10-19', 'region': 'fr'}
{'title': "De l'autre côté (feat. Nekfeu)", 'artist': 'Ninho', 'streams': 120611, 'id': '3ldfGgw7IoLHbaqmspX3ME', 'date': '2017-10-19', 'region': 'fr'}
{'title': 'Basique', 'artist': 'Orelsan', 'streams': 118592, 'i

In [111]:
with open("FR-2017-10-19.txt", "a+") as f:
    for song in data:
        f.write(str(song) + "\n")

#### The below cell doesn't really create valid JSON - it's 200 JSONs appended in a file...

In [103]:
import json
with open("FR-2017-10-19.json", "a+") as f:
    for song in data:
        json.dump(song, f)